# Import Packages

In [0]:
import keras
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.optimizers import Adam

import numpy as np

import os
import matplotlib.pyplot as plt

from google.colab import files


# Model Variables

In [0]:
imageRows = 28
imageColumns = 28  
imageChannels = 1

imageShape = (imageRows, imageColumns, imageChannels)

imageLatentDimension = 100

optimizer = Adam(0.0002, 0.5)

# Models 

**Generator Model**

In [0]:
def generator():
  
  model = Sequential()
  
  model.add(Dense(128 * 7 * 7, activation = 'relu', input_dim = imageLatentDimension))
  model.add(Reshape((7, 7, 128)))
  
  model.add(UpSampling2D())
  model.add(Conv2D(128, kernel_size = 3, padding = 'same'))
  model.add(BatchNormalization(momentum = 0.8))
  model.add(Activation("relu"))
  
  model.add(UpSampling2D())
  model.add(Conv2D(64, kernel_size = 3, padding = "same"))
  model.add(BatchNormalization(momentum = 0.8))
  model.add(Activation("relu"))
  
  model.add(Conv2D(imageChannels, kernel_size = 3, padding = "same"))
  model.add(Activation("tanh"))
  
  model.summary()
  
  randomNoise = Input(shape = (imageLatentDimension, ))
  generatedImage = model(randomNoise)

  return Model(inputs = randomNoise, outputs = generatedImage)

**Discriminator Model**

In [0]:
def discriminator():

  model = Sequential()

  model.add(Conv2D(32, kernel_size = 3, strides = 2, input_shape = imageShape, padding = "same"))
  model.add(LeakyReLU(alpha = 0.2))
  model.add(Dropout(0.25))
  
  model.add(Conv2D(64, kernel_size = 3, strides = 2, padding = "same"))
  model.add(ZeroPadding2D(padding = ((0, 1), (0, 1))))
  model.add(BatchNormalization(momentum = 0.8))
  model.add(LeakyReLU(alpha = 0.2))
  model.add(Dropout(0.25))
  
  model.add(Conv2D(128, kernel_size = 3, strides = 2, padding = "same"))
  model.add(BatchNormalization(momentum = 0.8))
  model.add(LeakyReLU(alpha = 0.2))
  model.add(Dropout(0.25))
  
  model.add(Conv2D(256, kernel_size = 3, strides = 1, padding = "same"))
  model.add(BatchNormalization(momentum = 0.8))
  model.add(LeakyReLU(alpha = 0.2))
  model.add(Dropout(0.25))
  
  model.add(Flatten())
  model.add(Dense(1, activation = 'sigmoid'))

  model.summary()

  inputImage = Input(shape = imageShape)
  validity = model(inputImage)

  return Model(inputs = inputImage, outputs = validity)

# Driver Code

In [47]:
# Visualise Models
print ("Discriminator Model")
discriminatorModel = discriminator()

print ("\n\nGenerator Model")
generatorModel = generator()

# Compile Discrimiator Model
discriminatorModel.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])

# Generator Takes Random Noise As Input, Ergo Create Placeholder
randomNoise = Input(shape=(imageLatentDimension, ))
generatedImage = generatorModel(randomNoise)

# For The Final Combined Model, Train Only Generator
discriminatorModel.trainable = False

# Discriminator Takes Generated Images And Computes Validity
validity = discriminatorModel(generatedImage)

# Combined Model
combinedModel = Model(inputs = randomNoise, outputs = validity)
combinedModel.compile(optimizer = optimizer, loss = 'binary_crossentropy')

Discriminator Model
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_64 (Conv2D)           (None, 14, 14, 32)        320       
_________________________________________________________________
leaky_re_lu_41 (LeakyReLU)   (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_41 (Dropout)         (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_65 (Conv2D)           (None, 7, 7, 64)          18496     
_________________________________________________________________
zero_padding2d_11 (ZeroPaddi (None, 8, 8, 64)          0         
_________________________________________________________________
batch_normalization_47 (Batc (None, 8, 8, 64)          256       
_________________________________________________________________
leaky_re_lu_42 (LeakyReLU)   (None, 8, 8, 64)          0

# Train 

In [0]:
def train(epochs, batchSize = 128, saveInterval = 50):

  # Shape : (60000, 28, 28) 
  (X_train, _), (_, _) = mnist.load_data()

  # Rescale -1 To 1
  X_train = X_train / 127.5 - 1.
  
  # Expand Final Dimension To Include Channel (Shape Is Now (60000, 28, 28, 1))
  X_train = np.expand_dims(X_train, axis = 3)
  
  # Ground Truths For Adversarial Network
  real = np.ones((batchSize, 1))
  fake = np.zeros((batchSize, 1))
  
  for epoch in range(epochs):
    
      # ----------------------------------

      #         Train Discriminator

      # ----------------------------------
    
      # Select Random Batch Of Images
      imageIndices = np.random.randint(0, X_train.shape[0], batchSize)
      imagesToTrain = X_train[imageIndices]

      # Sample Noise And Generate Images
      randomNoise = np.random.normal(0, 1, (batchSize, imageLatentDimension))
      generatedImages = generatorModel.predict(randomNoise)
      
      # Train Discriminator By Classifying Real As 1's and Generated As 0's
      discriminatorLossReal = discriminatorModel.train_on_batch(imagesToTrain, real)
      discriminatorLossFake = discriminatorModel.train_on_batch(generatedImages, fake)
      discriminatorLoss = 0.5 * np.add(discriminatorLossReal, discriminatorLossFake)
    
      # ----------------------------------

      #         Train Generator

      # ----------------------------------

    
      generatorLoss = combinedModel.train_on_batch(randomNoise, real)
      
      # Print Progress
      print ("%d [D Loss: %f, Accuracy: %.2f%%] [G Loss: %f]" % (epoch, discriminatorLoss[0], 100 * discriminatorLoss[1], generatorLoss))
  
      # Save Images
      if epoch % saveInterval == 0:

        saveImages(epoch)
  


In [0]:
def saveImages(epoch):
       
        r, c = 5, 5
        randomNoise = np.random.normal(0, 1, (r * c, imageLatentDimension))
        generatedImages = generatorModel.predict(randomNoise)

        # Rescale Images From 0 - 1
        generatedImages = 0.5 * generatedImages + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i, j].imshow(generatedImages[cnt, :, :, 0], cmap = 'gray')
                axs[i, j].axis('off')
                cnt += 1
        fig.savefig("mnist_%d.png" % epoch)
        plt.close()


In [0]:
train(epochs = 4000, batchSize = 32, saveInterval = 500)

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:975: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D Loss: 0.755006, Accuracy: 45.31%] [G Loss: 1.044996]
1 [D Loss: 0.806790, Accuracy: 46.88%] [G Loss: 1.024639]
2 [D Loss: 0.675313, Accuracy: 64.06%] [G Loss: 0.998591]
3 [D Loss: 0.710563, Accuracy: 56.25%] [G Loss: 1.077123]
4 [D Loss: 0.647560, Accuracy: 59.38%] [G Loss: 0.917193]
5 [D Loss: 0.639792, Accuracy: 59.38%] [G Loss: 1.032955]
6 [D Loss: 0.694599, Accuracy: 56.25%] [G Loss: 1.134694]
7 [D Loss: 0.741457, Accuracy: 46.88%] [G Loss: 1.086693]
8 [D Loss: 0.712962, Accuracy: 56.25%] [G Loss: 1.204743]
9 [D Loss: 0.730470, Accuracy: 48.44%] [G Loss: 1.093904]
10 [D Loss: 0.810671, Accuracy: 39.06%] [G Loss: 0.973493]
11 [D Loss: 0.797530, Accuracy: 45.31%] [G Loss: 0.914425]
12 [D Loss: 0.683043, Accuracy: 64.06%] [G Loss: 1.024422]
13 [D Loss: 0.816912, Accuracy: 32.81%] [G Loss: 0.959739]
14 [D Loss: 0.697374, Accuracy: 53.12%] [G Loss: 0.972126]
15 [D Loss: 0.692412, Accuracy: 60.94%] [G Loss: 1.084098]
16 [D Loss: 0.722442, Accuracy: 56.25%] [G Loss: 0.858950]
17 [D L

In [0]:
for number in range(0, 4000, 100):
  file = 'mnist_' + str(number) + '.png'
  
  try :
    files.download(file)
  except :
    None
    
print ("Images Downloaded To Local Drive")